In [60]:
!pip install tensorflow  opencv-python mediapipe sklearn matplotlib pillow

In [84]:
pip install pysocks scikit-learn  

In [92]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp
from PIL import Image, ImageDraw, ImageFont
import datetime

In [86]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
import tensorflow as tf

In [87]:
# Path for exported data, numpy arrays
DATA_PATH = os.path.join('MP_Data') 
current_directory = os.getcwd()
# Actions that we try to detect
# actions = np.array(['අ', 'ආ', 'ඇ','ඈ','ඉ','ඊ','උ','ඌ','එ','ඒ','ඔ','ඕ','ක්','ග්','ජ්','ට්','ද්','ණ්','ත්','ඩ්','න්','ප්','බ්','ම්','ය්','ර්','ල්','ව්','ස්','හ්'])
actions = np.array(['අ', 'ආ', 'ඇ','ඈ','ඉ'])				
# Thirty videos worth of data
no_sequences = 30

# Each One Video  going to be 30 frames in length
sequence_length = 30

# Folder start
start_folder = 0

In [88]:
label_map = {label:num for num, label in enumerate(actions)}

In [89]:
sequences, labels = [], []
# loop each action
for action in actions:
    # Loop through the each action sequence(videos)
    for sequence in np.array(os.listdir(os.path.join(DATA_PATH, action))).astype(int):
        # this array will hold all the frame in one video
        window = []
        # Loop though the each frame 30(Frame)
        for frame_num in range(sequence_length):
            # using np.load function load the saved np array from folder (path,which action,which video number,which frame number)
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            # add loaded framee to window
            window.append(res)
        #    
        sequences.append(window)
        labels.append(label_map[action])

In [90]:
# shape should be 90 videos,30 frame each,1353 key points in a frame
np.array(sequences).shape

(150, 30, 126)

In [68]:
np.array(labels).shape

(150,)

In [69]:
X = np.array(sequences)

In [70]:
y = to_categorical(labels).astype(int)

In [71]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

In [72]:
from tensorflow.keras.models import Sequential
# Make us able to action detection
from tensorflow.keras.layers import LSTM, Dense
# Make us able to monitor our modle during taining
from tensorflow.keras.callbacks import TensorBoard

In [95]:
# Get the current date in YYYYMMDD format
current_date_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

checkpoint_path = f"checkpoints/cp-{current_date_time}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Check if the checkpoint directory exists, if not, create it
if not os.path.exists(checkpoint_dir):
    os.makedirs(checkpoint_dir)

# Check if a latest checkpoint exists
latest_checkpoint = tf.train.latest_checkpoint(checkpoint_dir)


cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path,  # Path to save the model weights
    save_weights_only=True,     # Save only the model weights
    verbose=1,                  # Verbosity mode, 1 for progress bar
    save_freq=50          
)



In [96]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [97]:
# load Sequential api
model = Sequential()
# add 3 layers of lstm model
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,126)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [98]:
# If a latest checkpoint exists, load the weights and continue training
if latest_checkpoint:
    print(f"Loading weights from checkpoint: {latest_checkpoint}")
    model.load_weights(latest_checkpoint)
# else:
#     if os.path.exists('action.h5'):
#         # Load the pre-trained model from the H5 file
#         model = tf.keras.models.load_model('action.h5')

#         # Get the model summary
#         model.summary()

        # # Get the model weights
        # model_weights = loaded_model.get_weights()
        # model = model.set_weights(model_weights)



Loading weights from checkpoint: checkpoints\cp-1550.ckpt


In [99]:
# Evaluate the model on the test set
loss, accuracy = model.evaluate(X_test, y_test)

# Print the accuracy
print(f"Test Accuracy: {accuracy:.2f}")

1/1 [==============================] - 1s 812ms/step - loss: 0.0169 - categorical_accuracy: 1.0000
Test Accuracy: 1.00


In [100]:
model.fit(X_train, y_train, epochs=2000, callbacks=[tb_callback,cp_callback])

Epoch 1/2000
5/5 [==============================] - 4s 28ms/step - loss: 3.3328e-05 - categorical_accuracy: 1.0000
Epoch 2/2000
5/5 [==============================] - 0s 27ms/step - loss: 3.3094e-05 - categorical_accuracy: 1.0000
Epoch 3/2000
5/5 [==============================] - 0s 26ms/step - loss: 3.2909e-05 - categorical_accuracy: 1.0000
Epoch 4/2000
5/5 [==============================] - 0s 26ms/step - loss: 3.2718e-05 - categorical_accuracy: 1.0000
Epoch 5/2000
5/5 [==============================] - 0s 27ms/step - loss: 3.2553e-05 - categorical_accuracy: 1.0000
Epoch 6/2000
5/5 [==============================] - 0s 27ms/step - loss: 3.2363e-05 - categorical_accuracy: 1.0000
Epoch 7/2000
5/5 [==============================] - 0s 27ms/step - loss: 3.2170e-05 - categorical_accuracy: 1.0000
Epoch 8/2000
5/5 [==============================] - 0s 27ms/step - loss: 3.1996e-05 - categorical_accuracy: 1.0000
Epoch 9/2000
5/5 [==============================] - 0s 26ms/step - loss: 3.1794e

KeyboardInterrupt: 

In [101]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_12 (LSTM)              (None, 30, 64)            48896     
                                                                 
 lstm_13 (LSTM)              (None, 30, 128)           98816     
                                                                 
 lstm_14 (LSTM)              (None, 64)                49408     
                                                                 
 dense_12 (Dense)            (None, 64)                4160      
                                                                 
 dense_13 (Dense)            (None, 32)                2080      
                                                                 
 dense_14 (Dense)            (None, 5)                 165       
                                                                 
Total params: 203525 (795.02 KB)
Trainable params: 203

In [102]:
res = model.predict(X_test)

1/1 [==============================] - 0s 347ms/step


In [103]:
actions[np.argmax(res[4])]

'ඈ'

In [104]:
actions[np.argmax(y_test[4])]

'ඈ'

In [105]:
model.save('action.keras')